In [20]:
import pandas as pd

In [21]:
merged_df = pd.read_csv('finaldata/merged_df.csv')

In [22]:
merged_df.drop(columns=merged_df[['PM2.5', 'CO', 'SO2', 'PN']], inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# Including 'Street' in the subset for plotting
merged_df['Datum'] = pd.to_datetime(merged_df['Datum'])
time_series_data_with_street = merged_df[['Datum', 'Street', 'TotalDailyTraffic', 'NO2', 'PM10']]

# Function to plot time series for each unique street
def plot_time_series_for_streets(data, column, title, color):
    streets = data['Street'].unique()
    n_streets = len(streets)

    # Creating subplots for each street
    fig, axs = plt.subplots(n_streets, 1, figsize=(15, 4 * n_streets))

    for i, street in enumerate(streets):
        street_data = data[data['Street'] == street]
        axs[i].plot(street_data['Datum'], street_data[column], label=street, color=color)
        axs[i].set_title(f'{title} for {street}')
        axs[i].set_xlabel('Date')
        axs[i].set_ylabel(column)
        axs[i].legend()
        axs[i].xaxis.set_major_locator(mdates.YearLocator())
        axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    # Adjusting layout
    plt.tight_layout()

    # Showing the plots
    plt.show()

# Plotting time series for 
plot_time_series_for_streets(time_series_data_with_street, 'TotalDailyTraffic', 'Total Daily Traffic', 'blue')
plot_time_series_for_streets(time_series_data_with_street, 'NO2', 'NO2 Levels', 'red')
plot_time_series_for_streets(time_series_data_with_street, 'PM10', 'PM10 Levels', 'green')


In [42]:
meteo_dt = pd.read_csv('finaldata/meteo_dt.csv')

In [ ]:
merged_meteo = pd.merge(merged_df, meteo_dt, how="left", on=['Datum', 'Street'])

In [47]:
merged_meteo = pd.read_csv('finaldata/merged_meteo.csv')
merged_meteo.drop(columns=merged_meteo[['Unnamed: 0', 'Year']], inplace=True)
merged_meteo['Datum'] = pd.to_datetime(merged_meteo['Datum']).dt.date

# Define a function to determine the season for a given date
def get_season(date):
    month = date.month
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        # December to February
        return 'Winter'

# Apply the function to the 'Datum' column to create a new 'Season' column
merged_meteo['Season'] = merged_meteo['Datum'].apply(get_season)

# Create dummy variables for the 'Season' column
season_dummies = pd.get_dummies(merged_meteo['Season'], prefix='Season')

# Concatenate the dummy variables with the original DataFrame
merged_meteo = pd.concat([merged_meteo, season_dummies], axis=1).drop(columns=['Season'])

In [48]:
#merged_meteo.to_csv('finaldata/merged_with_dummies.csv')